In [1]:
from graphs import *

In [2]:
config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 500,
    'width': 700,
    'scale': 7  }} # Multiply title/legend/axis/canvas sizes by this factor


In [3]:
unrate_usa()
cpi_usa()
fedfunds_usa()
vix()
spreads_usa()
savings_usa()
sp_tsx()
sp_500()

In [4]:
sp_tsx()